In [139]:
import numpy as np
import pandas as pd
import os
import glob
import shutil
import random
import mplcursors
import matplotlib.pyplot as plt
import networkx as nx
from ipywidgets import interactive
import ipywidgets as widgets
from IPython.display import display, clear_output

from GridDataGen.utils.io import *
from GridDataGen.utils.process_network import *
from GridDataGen.utils.config import *
from GridDataGen.utils.stats import *
from GridDataGen.utils.param_handler import *
from GridDataGen.utils.load import *
from GridDataGen.utils.solvers import *
from pandapower.auxiliary import pandapowerNet

import pandapower as pp
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

%matplotlib ipympl


### Case300

In [232]:
grid_name = "case300"
net = load_net_from_pp(grid_name)

network_preprocessing(net)
net.load['p_mw'] = net.load['p_mw']*0.8
net.load['q_mvar'] = net.load['q_mvar']*0.8
opf_converged = run_opf(net)
net = pf_preprocessing(net)
pf_converged = run_pf(net)

# Save this network to disk in the pf_node.csv and pf_edge.csv formats
root_path = "./temp"
os.makedirs(root_path, exist_ok=True)
node_path = os.path.join(root_path, "pf_node.csv")
edge_path = os.path.join(root_path, "pf_edge.csv")

node_data = pf_post_processing(net)
edge_data = get_adjacency_list(net)

try:
    os.remove(node_path)
    os.remove(edge_path)
except OSError:
    pass

save_node_edge_data(net, node_path, edge_path, node_data, edge_data)

# read node and edge data from csv
edge_df = pd.read_csv(edge_path)
node_df = pd.read_csv(node_path)

shutil.rmtree(root_path)

gen vm_pu > bus max_vm_pu for gens [18 27 28]. Setting bus limit for these gens.
gen vm_pu < bus min_vm_pu for gens [22]. Setting bus limit for these gens.


In [234]:
global G
G = nx.Graph()


# using conductance (G) as an example of a line-variable to visualize for now
edges = [
        (u, v) # this random value should come from the contingency data
        for u, v, g in zip(
            edge_df["index1"].tolist(), edge_df["index2"].tolist(), edge_df["G"].tolist()
        )
        if u != v
    ]
G.add_edges_from(edges)

global g_vals
global b_vals
g_vals = [
        g # this random value should come from the contingency data
        for u, v, g in zip(
            edge_df["index1"].tolist(), edge_df["index2"].tolist(), edge_df["G"].tolist()
        )
        if u != v
    ]

b_vals = [
        b # this random value should come from the contingency data
        for u, v, b in zip(
            edge_df["index1"].tolist(), edge_df["index2"].tolist(), edge_df["B"].tolist()
        )
        if u != v
    ]
#G.add_edges_from(edges)

# Set node positions
pos = nx.spring_layout(G, seed=4)
#pos = nx.random_layout(G, seed=4)
#pos = nx.planar_layout_layout(G, seed=42)

# Assign labels based on node type
node_shapes = {"REF": "s", "PV": "H", "PQ": "o"}
num_nodes = node_df.shape[0]
mask_PQ = node_df.loc[:, "PQ"] == 1
mask_PV = node_df.loc[:, "PV"] == 1
mask_REF = node_df.loc[:, "REF"] == 1
node_labels = {}
for i in range(num_nodes):
    if mask_REF[i]:
        node_labels[i] = "REF"
    elif mask_PV[i]:
        node_labels[i] = "PV"
    elif mask_PQ[i]:
        node_labels[i] = "PQ"

num_nodes = len(node_df)

In [235]:
g_vals

[0.0,
 -27.027027027027025,
 0.0,
 0.0,
 -12.19512195121951,
 -7.843137254901962,
 -4.669406051550243,
 0.0,
 0.0,
 0.0,
 0.0,
 -1.658031088082902,
 -20.000000000000007,
 0.0,
 0.0,
 -5.479452054794521,
 -27.027027027027025,
 0.0,
 -6.84150513112885,
 -12.19512195121951,
 0.0,
 0.0,
 0.0,
 0.0,
 -12.19512195121951,
 -20.000000000000007,
 -7.843137254901962,
 -3.504751634427445,
 -6.725297465080186,
 -6.84150513112885,
 -7.843137254901962,
 0.0,
 0.0,
 -3.504751634427445,
 -7.843137254901962,
 0.0,
 -6.557377049180326,
 0.0,
 -12.19512195121951,
 0.0,
 -8.733624454148472,
 0.0,
 -6.557377049180326,
 -5.033557046979865,
 -6.725297465080186,
 -7.142857142857141,
 -7.142857142857141,
 -0.7127101158510295,
 -15.10212684443395,
 -0.9889090051575404,
 -1.388401935348152,
 -0.680923502500266,
 -5.479452054794521,
 -0.7127101158510295,
 -4.098360655737705,
 -15.10212684443395,
 0.0,
 -1.658031088082902,
 -6.097560975609755,
 -1.9789922362612264,
 -5.033557046979865,
 0.0,
 -11.895910780669146,


In [ ]:
def update_graph(G, line_idx):
    """
    """
    
    edges, weights = zip(*nx.get_edge_attributes(G,'weight').items())
    edges = [(u, v, random.randint(0, 1)) for u, v in edges]
    G.add_weighted_edges_from(edges)

    return G


def animate_edges(
    G, pos, num_nodes, node_labels, cmap, line_idx, grid_line_var
):
    """
    """
    #print(f"line_idx:", line_idx, type(line_idx))
    node_shapes = {"REF": "s", "PV": "H", "PQ": "o"}

    fig, ax = plt.subplots(1, 2, figsize=(18, 7))
    mplcursors.cursor(fig, hover=True)
    # Third plot (predicted values without masking)
    for node_type, shape in node_shapes.items():
        nodes = [i for i in node_labels if node_labels[i] == node_type]
        node_size = 15 if node_type == "REF" else 7
        nx.draw_networkx_nodes(
            G,
            pos,
            nodelist=nodes,
            #node_color=[predicted_values[i] for i in nodes],
            cmap=cmap,
            node_size=node_size,
            ax=ax[0],
            #vmin=vmin,
            #vmax=vmax,
            node_shape=shape,
        )
    #edges, weights = zip(*nx.get_edge_attributes(G,'weight').items())
    #color_map = ['red' if w == 1 else 'darkgreen' for w in weights]
    #labels = {(u, v) : round(random.uniform(0., 20.), 2) for u, v in edges}
    

    edge_colors = ["gray"] * len(G.edges())
    edge_widths = [1] * len(G.edges())
    edge_colors[line_idx] = "red"
    edge_widths[line_idx] = 3
    #if grid_line_var == "G":
    #    edge_colors = g_vals
    #elif grid_line_var == "B":
    #    edge_colors = b_vals
    #vmin = np.min(edge_colors)
    #vmax = np.max(edge_colors)        

    #nx.draw_networkx_edges(G, pos, edge_color=color_map, alpha=1.0, ax=ax, width=1)
    nx.draw_networkx_edges(G, 
                           pos, 
                           edge_color=edge_colors, 
                           #edge_color=edge_colors,
                           
                           #edge_cmap=plt.cm.jet,
                           #edge_vmin = vmin,
                           #edge_vmax = vmax,
                           #alpha=1.0, 
                           ax=ax[0], 
                           width=edge_widths)
                           #width=1)
    
    #nx.draw_networkx_edge_labels(G,
    #                             pos, 
    #                             edge_labels=labels, 
    #                             font_size=3)
    # nx.draw_networkx_labels(
    #     G,
    #     pos,
    #     labels=node_labels,
    #     font_size=10,
    #     font_color="white",
    #     font_weight="bold",
    #     ax=ax,
    # )
    ax[0].set_title(f"Case300 Contingency Analysis", fontsize=14, fontweight="bold")

    ax[1].hist(c_data[line_idx], bins=50)

    #for spine in ax.spines.values():
    #    spine.set_linewidth(2)  # Adjust thickness


    #plt.subplots_adjust(right=0.9)
    plt.show()
    #return ax

def f(line_idx, grid_line_var):
    #global G
    #for _ in range(n_cases):
    #clear_output()
    #G = update_graph(G, line_idx)
    animate_edges(G, pos, num_nodes, node_labels,  plt.cm.viridis, line_idx, grid_line_var)
    #display(ax_)




In [257]:
line_index_slider = widgets.SelectionSlider(
    options=list(np.arange(0, len(G.edges()))),
    value=1,
    description='Select Line',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

grid_line_var_dropdown = widgets.Dropdown(
    options=['G', 'B'],
    value='G',
    description='Line Var',
    disabled=False,
)


In [258]:
c_data = {}
global c_data
for lidx in range(0, len(G.edges())):
    data = np.random.normal(loc=0, scale=1, size=300)
    c_data[lidx] = data

In [259]:
#interactive_plot = widgets.interactive(f, n_cases=(1, 50, 5), top_k=['line 1', 'line 2'])
interactive_plot = widgets.interactive(f, 
                                       line_idx=line_index_slider, 
                                       grid_line_var=grid_line_var_dropdown)
#output = interactive_plot.children[-1]
#output.layout.height = '800px'
interactive_plot

interactive(children=(SelectionSlider(continuous_update=False, description='Select Line', index=1, options=(0,…